## Clip datasets to each Parish council

In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np
from pathlib import Path

In [4]:
# Get current working directory
cwd = Path.cwd()

In [5]:
# Read in parish council boundaries
gdf = gpd.read_file("G:/OS_OpenData/OS_BoundaryLine/OS_BL_Parish_Wilts.shp")
gdf = gdf.to_crs("EPSG:27700")
gdf['AreaPC'] = gdf['HECTARES']

In [ ]:
# Dictionary listing all files, and relevant metadata needed for processing
file_info_shp = {
     "Landscape Character Areas":{
        "file_name":"G:/Wiltshire_Council/Landscape_Character_Areas/LCA_Combined_2005.shp",
        "source":"WC",
        "group":"LCA",
                                            "group_field":"Wc_lct",
                                            "groupColumnName":"Area",
                                             "groupColumnValue":False,
                       
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA"},
    "Ancient Tree Inventory":{
        "file_name":"G:/Woodland_Trust/Woodland_Trust_AncientTreeInventoryATI_Wilts.shp",
        "source":"WT",
        "group":"Woodland",
                                            "group_field":False,
                                            "groupColumnName":"Total",
                                            "groupColumnValue":False,
                                             "unit":"COUNT",
                                            "unit_type":"COUNT",
                                            "count_col_name":"Id",},
    "Bedrock":{
        "file_name":"G:/British_Geological_Survey_OpenData/Bedrock Geology/625k_V5_BEDROCK_Geology_Polygons.shp",
        "source":"BGS",
       "group":"Geology",
                                            "group_field":"LEX_RCS_D",
                                            "groupColumnName":"Bedrock",
                                            "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},
    "Faultlines":{
        "file_name":"G:/British_Geological_Survey_OpenData/Fault Geology/625k_V5_FAULT_Geology_Lines_Wilts.shp",
        "source":"BGS",
       "group":"Geology",
                                            "group_field":"CATEGORY",
                                            "groupColumnName":"Faultline",
                                            "groupColumnValue":False,
                                            "unit":"Metres",
                                            "unit_type":"LENGTH",
                                            "area_col_name":"LENGTH",},
    "Lakes Inventory Councils":{
        "file_name":"G:/CEH/Lakes_Inventory/CEH_Lakes_Inventory_Wilts.shp",
        "source":"CEH",
        "group":"Water Features",
                                            "group_field":False,
                                            "groupColumnName":"Total",
                                            "groupColumnValue":False,
                                            "unit":"HECTARES",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},
    "Land Cover":{
        "file_name":"G:/CEH/CEH_LCM2023/gb2023lcm.sqlite",
        "source":"CEH",
        "group":"CEH Land Cover",
                                            "group_field":"_mode",
                                            "groupColumnName":"Land Cover",
                                            "groupColumnValue":False,
                                            "unit":"HECTARES",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},
     
    "Flood Alert Areas":{
        "file_name":"G:/EA/Flood_Alert_Areas_Wilts.shp",
        "source":"EA",
        "group":"Water Features",
                                            "group_field":False,
                                            "groupColumnName":"Total",
                                            "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},
    "Flood Warning Areas":{
        "file_name":"G:/EA/Flood_Warning_Areas_Wilts.shp",
        "source":"EA",
        "group":"Water Features",
                                            "group_field":False,
                                            "groupColumnName":"Total",
                                            "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},
      "Kennet and Avon Canals":{
        "file_name":"G:/Canals_Rivers_Trust/Canals/CRT_Kennet_Avon_Canal.shp",
        "source":"Canals Rivers Trust",
        "group":"Water Features",
                                            "group_field":False,
                                            "groupColumnName":"Total",
                                            "groupColumnValue":False,
                                            "unit":"Metres",
                                            "unit_type":"Length",
                                            "area_col_name":"Length",},
    "Kennet and Avon Canals Planning Buffer":{
        "file_name":"G:/Canals_Rivers_Trust/Canals/CRT_Kennet_Avon_Canal_Planning_Buffer_Wilts.shp",
        "source":"Canals Rivers Trust",
        "group":"Water Features",
                                            "group_field":False,
                                            "groupColumnName":"Total",
                                            "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},
    "Main Rivers":{
        "file_name":"G:/EA/Statutory_Main_Rivers_Wilts.shp",
        "source":"EA",
        "group":"Water Features",
                                            "group_field":False,
                                            "groupColumnName":"Total",
                                            "groupColumnValue":False,
                                            "unit":"Metres",
                                            "unit_type":"LENGTH",
                                            "area_col_name":"LENGTH",},  
     "Risk of Flooding":{
        "file_name":"G:/EA/Originals/Flooding/EA_RiskOfFloodingFromRiversAndSea/Risk_of_Flooding_from_Rivers_and_Sea_Wilts.shp",
        "source":"EA",
        "group":"Water Features",
                                            "group_field":False,
                                             "groupColumnName":"Total",
                                            "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},
    "National Forest Inventory":{
        "file_name":"G:/FC_OpenData/NFI_Woodland/FC_NFI_2023_Wilts.shp",
        "source":"FC",
        "group":"Woodland",
                                            "group_field":"IFT_IOA",
                                            "groupColumnName":"Category",
                                            "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},       


    "Greenspace":{
        "file_name":"G:/OS_OpenData/OS_OpenGreenspace/OS_Greenspace_Wilts.shp",
        "source":"OS",
        "group":"Greenspace",
                     "group_field":"function",
                     "groupColumnName":"Category",
                     "groupColumnValue":False,
                     "unit":"Ha",
                    "unit_type":"Area"},

    "Priority Habitat Inventory":{
         "file_name":"G:/NE_OpenData/PHI/NE_Priority_Habitat_Inventory_v3-11-22_Wilts.shp",
        "source":"NE",
        "group":"PHI",
                                            "group_field":"MainHabs",
                                            "groupColumnName":"Habitat",
                                            "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA"},  
    "WSBRC Priority Habitats":{
        "file_name":"G:/WSBRC_Data/Habitats/WiltsPriorityHabs_polys_repair.shp",
        "source":"WSBRC",
        "group":"WSBRC Priority Habitats",
                                            "group_field":"PH_Name",
                                            "groupColumnName":"Priority Habitat",
                                             "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},

    "Chalk Streams":{
         "file_name":"G:/EA/Chalk_Streams_Wilts.shp",
        "source":"EA",
        "group":"Water Features",
                                            "group_field":False,
                                            "groupColumnName":"Total",
                                            "groupColumnValue":False,
                                            "unit":"Metres",
                                            "unit_type":"LENGTH",
                                            "area_col_name":"LENGTH"},

"Long-Established Woodland and Wood Pasture":{
     "file_name":"O:/NaturalEngland/Ancient Woodland Inventory Update/_Deliverable/Final/AWI_methodology_layer.shp",
     "source":"WSBRC",
     "group":"Woodland",
                                            "group_field":"Status_LEW",
                                            "groupColumnName":"Category",
                                            "groupColumnValue":{"ASNW":"Ancient Semi Natural Woodland",
                                                                "AWP": "Ancient Wood Pasture",
                                                                "IAWP":"Infilled Ancient Wood Pasture",
                                                                "LEW":"Long Established Woodland",
                                                                "PAWS":"Plantation on Ancient Woodland Site"},
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},
"Landscape Biodiversity Areas":{
     "file_name":"G:/WSBRC_Data/BAP_LandscapeBiodiversityAreas/WSBRC_LandscapeBiodiversityAreas.shp",
     "source":"WSBRC",
    "group":"LBAs",
                                            "group_field":"LBA_Name",
                                            "groupColumnName":"Name",
                                            "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA"},

   "Sustrans National Cycle Network Route":{
     "file_name":"G:/Sustrans/National_Cycle_Network/Sustrans_National_Cycle_Network_Wilts.shp",
     "source":"Sustrans",
    "group":"Access and Routeways",
                                            "group_field":"Desc_",
                                            "groupColumnName":"Route type",
                                            "groupColumnValue":False,
                                            "unit":"Metres",
                                            "unit_type":"Length"},
    "Sustrans_Regional Cycle Route":{
     "file_name":"G:/Sustrans/Sustrans_Regional_Cycle_Routes.shp",
     "source":"Sustrans",
    "group":"Access and Routeways",
                     "group_field":"Descriptio",
                     "groupColumnName":"Route type",
                     "groupColumnValue":False,
                                            "unit":"Metres",
                                            "unit_type":"Length"},
    "Sustrans Reclassified Cycle Route":{ "file_name":"G:/Sustrans/Reclassified Routes/Sustrans_Reclassified_Routes_Wilts.shp",
     "source":"Sustrans",
    "group":"Access and Routeways",
                     "group_field":"Desc_",
                     "groupColumnName":"Route type",
                     "groupColumnValue":False,
                                            "unit":"Metres",
                                            "unit_type":"Length"},
    "Public Rights of Way":{"file_name":"G:/Wiltshire_Council/Access_PROW/WiltshirePROWJan2022.shp",
     "source":"WC",
    "group":"Access and Routeways",
                      "group_field":"LYR_NAME",
                      "groupColumnName":"Access type",
                      "groupColumnValue":False,
                                            "unit":"Metres",
                                            "unit_type":"Length"},
    "Forest Roads":{
        "file_name":"G:/FC_OpenData/Management/FC_Roads_Wilts.shp",
     "source":"FC",
     "group":"Access and Routeways",
                     "group_field":False,
                     "groupColumnName":"Total",
                                             "groupColumnValue":False,
                                            "unit":"Metres",
                                            "unit_type":"Length"},
    "CRoW S16 Dedicated Land":{
        "file_name":"G:/NE_OpenData/CROW/NE_CRoW_Section_16_Dedicated_Land_Wilts.shp",
     "source":"NE",
        "group":"Access and Routeways",
                     "group_field":False,
                      "groupColumnName":"Total",
                                             "groupColumnValue":False,
                     "unit":"Ha",
                    "unit_type":"Area"},
    "CRoW_S15 Land All Types":{
        "file_name":"G:/NE_OpenData/CROW/NE_CRoW_Section_15_Land_All_Types_Wilts.shp",
     "source":"NE",
     "group":"Access and Routeways",
                     "group_field":False,
                      "groupColumnName":"Total",
                                             "groupColumnValue":False,
                     "unit":"Ha",
                    "unit_type":"Area"},
    "Conclusive Registered Commons":{
        "file_name":"G:/NE_OpenData/CROW/NE_CRoW_Conclusive_Registered_Common_Land_Wilts.shp",
     "source":"NE","group":"Access and Routeways",
                     "group_field":False,
                      "groupColumnName":"Total",
                                             "groupColumnValue":False,
                     "unit":"Ha",
                    "unit_type":"Area"},
    "Conclusive Open Country":{
         "file_name":"G:/NE_OpenData/CROW/NE_CRoW_Conclusive_Open_Country_Wilts.shp",
     "source":"NE",
     "group":"Access and Routeways",
                     "group_field":False,
                      "groupColumnName":"Total",
                                             "groupColumnValue":False,
                     "unit":"Ha",
                    "unit_type":"Area"},
     "CRoW Access Land":{
          "file_name":"G:/NE_OpenData/CROW/NE_CRoW_Access_Land_Wilts.shp",
     "source":"NE",
     "group":"Access and Routeways",
                     "group_field":False,
                      "groupColumnName":"Total",
                                             "groupColumnValue":False,
                     "unit":"Ha",
                    "unit_type":"Area"},
                    
  "Conservation Areas":{"file_name":"G:/Historic_England/ConservationAreas/HE_Conservation_Areas_Wilts.shp",
     "source":"HE",
      "group":"Heritage",
                                            "group_field":False,
                                            "groupColumnName":"Total",
                                             "groupColumnValue":False,
                                            "unit":"HECTARES",
                                            "unit_type":"AREA",
                                            "area_col_name":"sum_area_h"},  
"Country Parks":{"file_name":"G:/NE_OpenData/Country_Parks/NE_Country_Parks_Wilts.shp",
     "source":"NE",
    "group":"Access and Routeways",
                                            "group_field":False,
                                            "groupColumnName":"Total",
                                             "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},
                                            
    "Crown Estates Land":{"file_name":"G:/Crown Estates/TCE_RegisteredTitle_2023Jul_Wiltshire.shp",
     "source":"NE",
        "group":"Other Land",
                                            "group_field":False,
                                            "groupColumnName":"Total",
                                             "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"AREA",
                                            "area_col_name":"AREA",},  
      "WWT reserves":{
     "file_name":"G:/WWT/WWT_Reserves_Public_201224/WiltshireWildlifeTrust_Reserves_Public_201224.shp",
     "source":"WSBRC",
     "group":"Protected Areas",
        "group_field":False,
        "groupColumnName":"Total",
        "groupColumnValue":False,
        "unit":"HECTARES",
        "unit_type":"AREA",  
        "area_col_name":"AREA"},  

    "Priority Rivers":{
          "file_name":"G:/NE_OpenData/PHI/Priority_River_Habitat_Wilts.shp",
     "source":"NE",
     "group":"Priority Habitat",
                                            "group_field":False,
                                            "groupColumnName":"Total",
        "groupColumnValue":False,
                                            "unit":"Metres",
                                            "unit_type":"LENGTH",
                                            "area_col_name":"LENGTH"},           

          "Priority Ponds":{
              "file_name":"G:/NE_OpenData/PHI/Surveyed_Priority_Ponds_Wilts.shp",
     "source":"NE",
     "group":"Priority Habitat",
                                            "group_field":False,
                                            "groupColumnName":"Total",
        "groupColumnValue":False,
                                            "unit":"COUNT",
                                            "unit_type":"COUNT",
                                            "count_col_name":"Site_name",},                  
     "Sites of Special Scientific Interest":{
              "file_name":"G:/NE_OpenData/SSSI/Original/Sites_of_Special_Scientific_Interest_(England)___Natural_England.shp",
     "source":"NE",
     "group":"Protected Areas",
                                            "group_field":False,
                                            "groupColumnName":"Total",
        "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"Area",},         
     "Special Areas of Conservation":{
              "file_name":"G:/NE_OpenData/SAC/NE_Special_Areas_of_Conservation_Wilts.shp",
     "source":"NE",
     "group":"Protected Areas",
                                            "group_field":False,
                                            "groupColumnName":"Total",
        "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"Area",},           
     "Special Protection Areas":{
              "file_name":"G:/NE_OpenData/SPA/NE_Special_Protection_Areas_iWilts.shp",
     "source":"NE",
     "group":"Protected Areas",
                                            "group_field":False,
                                            "groupColumnName":"Total",
        "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"Area",},     
     "National Trust Land Always Open":{
              "file_name":"G:/National_Trust/NT_Land_Always_Open_Wilts.shp",
     "source":"National Trust",
     "group":"eNGO Conservation Sites",
                                            "group_field":False,
                                            "groupColumnName":"Total",
        "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"Area",},  
        "National Trust Land Limited Access":{
              "file_name":"G:/National_Trust/NT_Land_Limited_Access_Wilts.shp",
     "source":"National Trust",
     "group":"eNGO Conservation Sites",
                                            "group_field":False,
                                            "groupColumnName":"Total",
        "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"Area",},         
    "RSPB Reserves":{
              "file_name":"G:/RSPB/RSPB_Reserves/RSPB_Reserve_Wilts.shp",
     "source":"RSPB",
     "group":"eNGO Conservation Sites",
                                            "group_field":False,
                                            "groupColumnName":"Total",
        "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"Area",},      
    "Woodland Trust Sites":{
              "file_name":"G:/Woodland_Trust/Woodland_Trust_Sites_Wilts.shp",
     "source":"Woodland Trust",
     "group":"eNGO Conservation Sites",
                                            "group_field":False,
                                            "groupColumnName":"Total",
        "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"Area",},       
    "Crown Estates":{
              "file_name":"G:/Crown Estates/TCE_RegisteredTitle_2023Jul_Wiltshire.shp",
     "source":"Crown Estates",
     "group":"Other Land",
                                            "group_field":False,
                                            "groupColumnName":"Total",
        "groupColumnValue":False,
                                            "unit":"Ha",
                                            "unit_type":"Area",},     
     "Listed Buildings":{
              "file_name":"G:/Historic_England/ListedBuildings/HEListed_Building_poly_Wilts.shp",
     "source":"Historic England",
     "group":"Heritage",
                                            "group_field":False,
                                            "groupColumnName":"Total",
        "groupColumnValue":False,
                                            "unit":"Count",
                                            "unit_type":"Count",
                                            "count_col_name":"Name"},     
    "Scheduled Monuments":{
              "file_name":"G:/Historic_England/ScheduledMonuments/HE_Scheduled_Monuments_Wilts.shp",
     "source":"Historic England",
     "group":"Heritage",
                                            "group_field":False,
                                            "groupColumnName":"Total",
        "groupColumnValue":False,
                                            "unit":"Count",
                                            "unit_type":"Count",
                                            "count_col_name":"Name"},       
     "Registered Parks and Gardens":{
              "file_name":"G:/Historic_England/ParksGardens/HE_Registered_Parks_and_Gardens_Wilts.shp",
     "source":"Historic England",
     "group":"Heritage",
                                            "group_field":False,
                                            "groupColumnName":"Total",
        "groupColumnValue":False,
                                            "unit":"Count",
                                            "unit_type":"Count",
                                            "count_col_name":"Name"},     
    "World Heritage Sites":{
              "file_name":"G:/Historic_England/WorldHeritageSites/HE_World_Heritage_Sites_Wilts.shp",
     "source":"Historic England",
     "group":"Heritage",
                                            "group_field":False,
                                            "groupColumnName":"Total",
        "groupColumnValue":False,
                                            "unit":"Count",
                                            "unit_type":"Count",
                                            "count_col_name":"Name"},     
     "Battlefields":{
              "file_name":"G:/Historic_England/Battlefields/Registered_Battlefields_Wilts.shp",
     "source":"Historic England",
     "group":"Heritage",
                                            "group_field":False,
                                            "groupColumnName":"Total",
        "groupColumnValue":False,
                                            "unit":"Count",
                                            "unit_type":"Count",
                                            "count_col_name":"Name"},    
     "Conservation areas":{
              "file_name":"G:/Historic_England/ConservationAreas/HE_Conservation_Areas_Wilts.shp",
     "source":"Historic England",
     "group":"Heritage",
                                            "group_field":False,
                                            "groupColumnName":"Total",
        "groupColumnValue":False,
                                            "unit":"Count",
                                            "unit_type":"Count",
                                            "count_col_name":"NAME"},                                                                                                                                                                                                                                                                                                        
}                                                                                                                                                                                                                                                                                                      


In [7]:
   
   
### To do
## List
# LNR (local nature reserve)
# NNR (national nature reserve)
# RPG ()
# SAM
# WHS
# Forestry commission sites
# wiltshire council sites
# MOD SPTA 
# Living churchyard

## Files
#     # "Traditional Orchards":{
#     #     "file_name":"G:\NE_OpenData\PHI\NE_Priority_Habitat_Inventory_v3-11-22_Wilts.shp",
#     #     "source":"NE",
#     #     "group":"Priority Habitat",
#     #                                         "group_field":False,
#     #                                         "unit":"HECTARES",
#     #                                         "unit_type":"AREA",
#     #                                         "area_col_name":"AREA"},  



#      "NE_Woodpasture_Parkland_Councils":{"group":"Priority Habitat",
#                                             "group_field":False,
#                                             "unit":"HECTARES",
#                                             "unit_type":"AREA",
#                                             "area_col_name":"AREA"},
#     # "NFI_AT":{"group":"TBC",
#     #                                         "group_field":"IFT_IOA",
#     #                                         "unit":"HECTARES",
#     #                                         "unit_type":"AREA",
#     #                                         "area_col_name":"Area_ha"},
#     "Protected_Landscapes_Councils":{"group":"Protected Landscape",
#                                             "group_field":False,
#                                             "unit":"HECTARES",
#                                             "unit_type":"AREA",
#                                             "area_col_name":"AREA"},  
#      "RCN_Councils":{"group":"Access and Routeways",
#                                             "group_field":False,
#                                             "unit":"Metres",
#                                             "unit_type":"LENGTH",
#                                             "area_col_name":"LENGTH"},       
# #     "RIGS_Councils":{"group":"Geology",
# #                                             "group_field":False,
# #                                             "unit":"Metres",
# #                                             "unit_type":"LENGTH",
# #                                             "area_col_name":"LENGTH"},
  
# 
#     # "Hedgerows_Councils":{"group":"Priority Habitat",
    #                                         "group_field":False,
    #                                         "unit":"Metres",
    #                                         "unit_type":"LENGTH",
    #                                         "area_col_name":"LENGTH"},
    # "HE_WHS_Count_Councils":{"group":"TBC",
    #                                         "group_field":False,
    #                                         "unit":"COUNT",
    #                                         "unit_type":"COUNT",
    #                                         "area_col_name":"COUNT",},   
    #  # "HLC_Councils":{"group":"TBC",
    #  #                                        "group_field":False,
    #  #                                        "unit":"HECTARES",
    #  #                                        "unit_type":"AREA",
    #  #                                        "area_col_name":"AREA",},
    # "LCA_Councils":{"group":"Landscape Character",
    #                                         "group_field":"Wc_lct",
    #                                         "unit":"HECTARES",
    #                                         "unit_type":"AREA",
    #                                         "area_col_name":"AREA",},       
    # "MOD_Land_Councils":{"group":"Other Land",
    #                                         "group_field":False,
    #                                         "unit":"HECTARES",
    #                                         "unit_type":"AREA",
    #                                         "area_col_name":"AREA"},
    # "NCN_Councils":{"group":"Access and Routeways",
    #                                         "group_field":False,
    #                                         "unit":"Metres",
    #                                         "unit_type":"LENGTH",
    #                                         "area_col_name":"LENGTH"}, 

In [ ]:
# Create empty dataframe to fill
df_all_shp = pd.DataFrame({'NAME': [], 'value': [],'groupColumnValue': [],'groupColumnName': [],'mapGroup': [], "unitName":[],"unit":[],"datasetName":[], "source":[]})

# Loop through each dataset listed in dictionary
for file in file_info_shp.keys():
    gdf_data = gpd.read_file(file_info_shp[file]["file_name"])
    gdf_data = gdf_data.to_crs("EPSG:27700")
    print(file)

    # Create empty dataframe to fill for each PC
    df_local_shp = pd.DataFrame({'NAME': [], 'value': [],'groupColumnValue': [],'groupColumnName': [],'mapGroup': [], "unitName":[],"unit":[],"datasetName":[], "source":[]})

    # Get stats for each PC
    for pc in gdf['NAME'].unique():
        # Filter parish council dataframe to relevant PC
        gdf_local = gdf[gdf.NAME == pc]
        # Clip data to PC
        gdf_clipped = gdf_data.clip(gdf_local)

        # If dataset is a count, just need to add all occurences
        if file_info_shp[file]["unit_type"].lower()=="count":
            gdf_dissolved = gdf_clipped[[file_info_shp[file]["count_col_name"],"geometry"]].dissolve( aggfunc="count")
            gdf_dissolved['value'] = gdf_dissolved[file_info_shp[file]["count_col_name"]]
        else:
            # If dataset is split by groups, need to summarise for each group
            if file_info_shp[file]["group_field"]:
                gdf_dissolved = gdf_clipped[[file_info_shp[file]["group_field"],"geometry"]].dissolve(by = file_info_shp[file]["group_field"], aggfunc="sum").reset_index()
            else:
                # If no groupings, can just summarise all polygons
                gdf_dissolved = gdf_clipped[["geometry"]].dissolve( aggfunc="sum").reset_index()
            
            # update to readable units by rounding
            if file_info_shp[file]["unit_type"].lower() == "length":
                gdf_dissolved['value'] = (gdf_dissolved.geometry.length).round(2)
            elif file_info_shp[file]["unit_type"].lower() == "area":
                gdf_dissolved['value'] = (gdf_dissolved.geometry.area/10000).round(2)

        # Update names of columns 
        gdf_dissolved['NAME'] = pc
        if file_info_shp[file]["group_field"]:
            gdf_dissolved['groupColumnValue'] = gdf_dissolved[file_info_shp[file]["group_field"]]
            if file_info_shp[file]["groupColumnValue"]:
                gdf_dissolved['groupColumnValue'] = gdf_dissolved['groupColumnValue'].map(file_info_shp[file]["groupColumnValue"])
            
        else:
            gdf_dissolved['groupColumnValue'] = "Total"

        
        # else:
        #     gdf_dissolved['groupColumnValue'] = "Total"
            
        gdf_dissolved['groupColumnName'] = file_info_shp[file]["groupColumnName"]
        gdf_dissolved['mapGroup'] = file_info_shp[file]["group"]
        gdf_dissolved['unitName'] = file_info_shp[file]["unit_type"]
        gdf_dissolved['unit'] = file_info_shp[file]["unit"]
        gdf_dissolved['datasetName'] = file
        gdf_dissolved['source'] = file_info_shp[file]["source"]

        gdf_dissolved = gdf_dissolved[['NAME', "value","mapGroup","groupColumnName","groupColumnValue","unitName","unit","datasetName", "source"]]

        df_local_shp = pd.concat([df_local_shp,gdf_dissolved ])
    # Output as individual Csv's so can be saved
    df_local_shp.to_csv(f"{cwd}/data/clippedData/{file}_v1.csv", index = False)    
    df_all_shp = pd.concat([df_all_shp,df_local_shp ])
    

Landscape Character Areas
